<a href="https://colab.research.google.com/github/kimdonggyu2008/hufs_hackerthon/blob/main/BBC_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [4]:
!pip install keras.preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

import wandb
import os
# print(os.listdir("/content/drive/MyDrive/코딩공부/dacon_project"))


In [6]:
# os.environ['WANDB_API_KEY']='513a1f0c050fa7f60a76b5232e904d8df397082e'
# os.environ['WANDB_ENTITY']='article classification'

In [7]:
# !wandb login

In [8]:
df=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')
# print("Shpe of Data",df.shape)
# df.head(10)

In [9]:
# df.info()

In [10]:
# print(df['분류'].value_counts())
# sns.countplot(data = df, x = "분류", palette = "RdBu")
# plt.title("The Distribution of Category")

In [11]:
# # Show the describe include object
# df.describe(include = 'object')

In [12]:
# # Lets calculate the length of the Reviews
# df['length'] = df['제목'].apply(len)

In [13]:
# plt.hist(x=df['length'])

#감정분석(필요?)

In [14]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.8 MB/s eta 0:00:00


In [15]:
!apt-get update -qq
!apt-get install -qq fonts-nanum

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [17]:
okt=Okt()

In [18]:
def preprocess_korean(text):
    # 특수문자 제거 및 소문자 변환
    text = re.sub(r'\W+', ' ', text)
    # 형태소 분석 후 명사만 추출
    tokens = okt.nouns(text)
    return ' '.join(tokens)

#카테고리를 번호로 치환

In [19]:
# 카테고리 목록과 매핑될 레이블 번호를 생성합니다.
categories = [
    "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
    "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
    "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
    "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
    "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
    "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
    "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
    "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
    "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
    "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
    "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
    "스포츠:야구", "경제:외환"
]

# DataFrame을 랜덤하게 섞습니다.
shuffled = df.reindex(np.random.permutation(df.index))

# 각 카테고리에 대해 데이터 샘플링
num_of_categories = 45000
sampled_data = []
for category in categories:
    sampled_data.append(shuffled[shuffled['분류'] == category][:num_of_categories])

# 샘플링된 데이터를 하나의 데이터프레임으로 합칩니다.
concated = pd.concat(sampled_data, ignore_index=True)

# 데이터프레임을 다시 섞습니다.
concated = concated.reindex(np.random.permutation(concated.index))

# LABEL 열 생성 (카테고리별 레이블 부여)
concated['라벨'] = concated['분류'].apply(lambda x: categories.index(x))

# One-hot 인코딩
labels = to_categorical(concated['라벨'], num_classes=len(categories))

# 카테고리 열 삭제
# if '분류' in concated.columns:
#     concated = concated.drop(['분류'], axis=1)

print(concated['라벨'][:10])
print(concated['분류'][:10])
# print(labels[:10])


25267     0
4245      0
37931     6
4809      0
42775    12
22080     0
52347    35
17950     0
40895     9
29653     1
Name: 라벨, dtype: int64
25267               지역
4245                지역
37931         정치:국회_정당
4809                지역
42775           경제:자동차
22080               지역
52347         IT_과학:과학
17950               지역
40895    스포츠:올림픽_아시안게임
29653           경제:부동산
Name: 분류, dtype: object


In [20]:
n_most_common_words = 20000
max_len = 150
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['제목'].values)
sequences = tokenizer.texts_to_sequences(concated['제목'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

Found 83611 unique tokens.


In [21]:
# print ("Shape of X:",len(X))

In [22]:
# from imblearn.over_sampling import SMOTE

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:

# 트랜스포머 블록 정의
def transformer_block(x, head_size, num_heads, ff_dim, dropout=0.3):
    # Multi-Head Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    attention_output = Dropout(dropout)(attention_output)

    # Add & Normalize
    attention_output = Add()([x, attention_output])
    attention_output = LayerNormalization()(attention_output)

    # Feed Forward Network
    ffn_output = Dense(ff_dim, activation='relu',kernel_regularizer=l2(0.001))(attention_output)
    ffn_output = Dropout(dropout)(ffn_output)
    ffn_output = Dense(emb_dim)(ffn_output)

    # Add & Normalize
    ffn_output = Add()([attention_output, ffn_output])
    ffn_output = LayerNormalization()(ffn_output)

    return ffn_output


In [ ]:
emb_dim = 128
# 입력층
input_layer = Input(shape=(max_len,))

# Embedding 레이어
embedding_layer = Embedding(input_dim=n_most_common_words, output_dim=emb_dim, input_length=max_len)(input_layer)


In [ ]:
# 기본 설정
n_most_common_words = 80000  # 어휘 수 80,000개로 설정
emb_dim = 128  # 임베딩 벡터의 차원 설정
max_len = X.shape[1]  # 시퀀스의 길이 (기사 제목의 길이)
num_heads = 8  # Multi-head attention에서 헤드의 개수
ff_dim = 128  # Feed Forward Network의 출력 차원
num_classes = 56  # 최종 클래스의 개수

# 최적화된 하이퍼파라미터
learning_rate = 0.0005        # 학습률 (조정 가능)
batch_size = 64             # 배치 크기
epochs = 10                  # 최대 에포크 수
dropout_rate = 0.7           # 드롭아웃 비율
l2_lambda = 0.001             # L2 정규화 값
patience = 7                  # 조기 종료를 위한 patience
min_delta = 0.001             # 조기 종료를 위한 최소 손실 감소량
validation_split = 0.2        # 검증 데이터 비율


In [ ]:
# 트랜스포머 블록 적용
x = transformer_block(embedding_layer, emb_dim, num_heads, ff_dim, dropout=dropout_rate)
#x = transformer_block(x, emb_dim, num_heads, ff_dim, dropout=dropout_rate)
# Global Average Pooling
x = GlobalAveragePooling1D()(x)

# 분류를 위한 Dense 레이어 (56개의 클래스)
output_layer = Dense(num_classes, activation='softmax')(x)

# 모델 정의


In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.summary()

In [ ]:
# wandb.init(
#     # W&B 프로젝트 이름을 설정
#     project="article classification",

#     # 하이퍼파라미터 및 실행 메타데이터를 추적
#     config={
#         "learning_rate": 0.001,
#         "architecture": "Transformer",
#         "dataset": "titles",
#         "epochs": 5
#         "max_sequence_length": 130,
#         "embedding_dim": 128,
#         "num_heads": 8,
#         "feedforward_dim": 128,
#         "num_classes": 56
#     }
# )

# """
# # 기본 설정
# n_most_common_words = 80000  # 어휘 수 80,000개로 설정
# emb_dim = 128  # 임베딩 벡터의 차원 설정
# max_len = X.shape[1]  # 시퀀스의 길이 (기사 제목의 길이)
# num_heads = 8  # Multi-head attention에서 헤드의 개수
# ff_dim = 128  # Feed Forward Network의 출력 차원
# num_classes = 56  # 최종 클래스의 개수
# """

In [ ]:
import matplotlib.pyplot as plt

# 학습 결과 시각화
def plot_training_history(history):
    # 정확도 그래프
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # 손실 그래프
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
# 컴파일 설정
optimizer=AdamW(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',     # 검증 손실을 모니터링
    mode='min',             # 'min' 모드로 설정하여, 손실이 감소하는 것을 추적
    patience=0,             # 손실이 더 이상 개선되지 않는 즉시 학습 중단
    restore_best_weights=True  # 가장 성능이 좋았던 가중치로 복원
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split,
    callbacks=[early_stopping])

Epoch 1/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 30s 42ms/step - accuracy: 0.4873 - loss: 2.6278 - val_accuracy: 0.4934 - val_loss: 2.7566
Epoch 2/10


In [ ]:
plot_training_history(history)

In [ ]:
# 모델 전체를 저장하는 코드 (모델 아키텍처, 가중치, 컴파일 정보 포함)
model.save('/content/drive/MyDrive/코딩공부/dacon_project/result/bbc_edit_model.h5')  # 모델을 'model.h5' 파일로 저장

print("모델이 'bbc_edit_model.h5'로 저장되었습니다.")


In [ ]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/train.csv')
# category_counts = df['분류'].value_counts()

# # 결과를 출력합니다.
# print(category_counts[:40])

In [ ]:
txt = ["용인특례시, 가을 신학기 학교 급식 공급업체 위생점검"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = [
    "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
    "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
    "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
    "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
    "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
    "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
    "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
    "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
    "경제:자원", "문화:종교", "IT_과학:콘텐츠", "사회:미디어", "사회:날씨",
    "스포츠:농구_배구", "문화:음악", "문화:생활", "IT_과학:보안", "스포츠:월드컵",
    "경제:증권_증시", "정치:북한", "정치:외교", "스포츠:스포츠일반", "문화:영화",
    "스포츠:야구", "경제:외환"
]
# labels = [ #40개 레이블만 있음
#     "지역", "경제:부동산", "사회:사건_사고", "경제:반도체", "사회:사회일반", "사회:교육_시험",
#     "정치:국회_정당", "사회:의료_건강", "경제:취업_창업", "스포츠:올림픽_아시안게임",
#     "경제:산업_기업", "문화:전시_공연", "경제:자동차", "경제:경제일반", "사회:장애인",
#     "스포츠:골프", "정치:선거", "경제:유통", "IT_과학:모바일", "사회:여성",
#     "사회:노동_복지", "사회:환경", "경제:서비스_쇼핑", "경제:무역", "정치:행정_자치",
#     "국제", "문화:방송_연예", "스포츠:축구", "경제:금융_재테크", "정치:청와대",
#     "문화:출판", "IT_과학:IT_과학일반", "IT_과학:인터넷_SNS", "문화:미술_건축",
#     "정치:정치일반", "IT_과학:과학", "문화:문화일반", "문화:학술_문화재", "문화:요리_여행",
#     "경제:자원"
# ]
# print(pred, labels[np.argmax(pred)])

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. test.csv 파일 로드
test_df = pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/test.csv')

# 2. 데이터 전처리: 제목을 시퀀스로 변환
# 학습 시 사용했던 tokenizer를 사용하여 제목을 시퀀스로 변환
sequences_test = tokenizer.texts_to_sequences(test_df['제목'].values)

# 3. 시퀀스의 길이를 동일하게 맞추기 (max_len에 맞춰 패딩)
X_test = pad_sequences(sequences_test, maxlen=max_len)

# 4. 모델을 사용하여 예측
predictions = model.predict(X_test)

# 5. 예측 결과에서 가장 확률이 높은 클래스를 추출
predicted_labels = predictions.argmax(axis=1)
predicted_labels[predicted_labels>45]=0

predicted_class_names=[categories[label] for label in predicted_labels]

# 6. submission 데이터프레임 생성 (id와 예측된 레이블을 저장)
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    '분류': predicted_class_names
})

# 7. submission.csv 파일로 저장
submission_df.to_csv('/content/drive/MyDrive/코딩공부/dacon_project/result/submission.csv', index=False)

print("Submission 파일이 생성되었습니다.")


In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/data/test.csv')
df_result=pd.read_csv('/content/drive/MyDrive/코딩공부/dacon_project/result/submission.csv')
print("Shpe of test Data",df_test.shape)
print("Shpe of test Data",df_result.shape)
# df_test.head(30)

In [ ]:
df_result.head(30)

In [ ]:
print(df_result['분류'].value_counts())


In [ ]:
print(df_result['분류'])